### Домашнее задание. Гибридные рекомендательные системы

1. Датасет ml-latest
2. Вспомнить подходы, которые мы разбирали
3. Выбрать понравившийся подход к гибридным системам
4. Написать свою

In [80]:
from surprise import KNNWithMeans, KNNBasic, SVD, BaselineOnly
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

для построения рекомендации будем использовать следующий подход: 
- делаем прогноз рейтинга на датасете для всех пользователей (используем SVD)
- определяем пользоватеся 
- формируем список фильмов, которые пользователь не смотрел
- сортируем по прогнозному рейтингу для этого пользователя, берем в работу 10 наилучших
- добавляем к потенциальным рейтингам пользователя, метрику средней оценки фильма, остальных пользователей
- создаем итоговую метрику, которая с весом 70% берет информацию о рейтинге из прогнозного пользователя и на 30% берет информацию о рейтинге из среднее данного другими пользователями
- отдаем в рекомендацию три фильма на основе финальной метрики

In [81]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [82]:
data = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
data.dropna(inplace=True)

строим модель для предсказания рейтинга пользователя

In [83]:
# задаем датасет в формате surprise
dataset = pd.DataFrame({
    'uid': data.userId,
    'iid': data.title,
    'rating': data.rating
})

In [84]:
reader = Reader(rating_scale=(0.5, 5.0))
data_surprise = Dataset.load_from_df(dataset, reader)

In [85]:
trainset, testset = train_test_split(data_surprise, test_size=.15, random_state=42)

In [86]:
# обучаем модель 
algo = SVD(n_factors=20, n_epochs=20)
algo.fit(trainset)

In [87]:
# делаем оценку качества на тесте
test_pred = algo.test(testset)

accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8689


0.8688912847922808

In [32]:
# оценка на кросс валидации
cros_val = cross_validate(algo, data_surprise, measures=['RMSE'], cv=5, verbose=True)
RMSE_mean = cros_val['test_rmse'].mean()

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8713  0.8725  0.8697  0.8618  0.8688  0.8688  0.0037  
Fit time          4.62    4.29    5.58    4.24    4.45    4.64    0.49    
Test time         0.24    1.50    0.22    0.26    0.42    0.53    0.49    


In [33]:
print(RMSE_mean)

0.8688230277116394


In [88]:
# определяем пользователя и фильмы которые он посмотрел
user = 7.0
user_movies = data[data.userId == user].title.unique()

In [89]:
# формируем списки по предсказанию и наименовнию фильма
pred_rating = []
titles = []

for movie in data.title.unique():
    if movie in user_movies:
        continue
        
    pred_rating.append(algo.predict(uid=user, iid=movie).est)
    titles.append(movie)

In [90]:
# база фильмов которые пользователь не смотрел и прогнозный рейтинг фильму от пользователя
movies_not_watch = pd.DataFrame({'title': titles, 'pred_r': pred_rating})

In [91]:
# берем топ 10 фильмом из потенциального прогноза пользователя
most_interesting = movies_not_watch.sort_values(by='pred_r', ascending=False)[:10]

In [92]:
# формируем базу фильмов с средним рейтингом по фактическим оценкам пользователя
movies_with_mean_rating = pd.DataFrame(data.groupby('title').rating.mean()).reset_index()

In [153]:
# задаем функцию которая добавит средний рейтинг
def add_mean(row):
    return movies_with_mean_rating[movies_with_mean_rating.title == row].rating.sum()

In [154]:
# в понтенциальные топ 10 добавляем фактический средний рейтинг
most_interesting['mean_rating'] = most_interesting.title.apply(add_mean)

In [156]:
# создаем метрику которая взвешивает два подхода на 70/30 процентов
most_interesting['common_rating'] = most_interesting.pred_r*0.7 + most_interesting.mean_rating*0.3

In [160]:
# формируем топ 3 рекомендации, на основе финальной метрики
recommend_for_user = most_interesting.sort_values(by='common_rating', ascending=False)[:3][['title', 'common_rating']]

In [161]:
recommend_for_user

,title,common_rating
881,Lawrence of Arabia (1962),4.237129
887,Goodfellas (1990),4.204081
806,Reservoir Dogs (1992),4.180371
